In [39]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re

In [40]:
data = pd.read_csv('movie_bd_v5.csv')

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [45]:
answers = {} # создадим словарь для ответов

data['profit'] = data['revenue'] - data['budget']

def create_list(record):
    '''Функция разделяет запись из датафрейма на составляющие элементы 
    и каждый элемент добавляет в список. Если элемент в записи только один, 
    то сразу добавляет в список.
    '''
    if '|' in record:
        for element in record.split('|'):
            record_list.append(element)
    else:
        record_list.append(record)

def create_list2 (record):
    '''Функция разделяет запись из датафрейма на составляющие элементы. 
    Затем для каждого элемента создает пару с другим элементом(не одинаковым) 
    и добавляет эту пару в список
    '''
    record = record.split('|')
    for element1 in record:
        for element2 in record:
            if element1 == element2 : continue
            element = element1 + ',' + ' ' + element2    
            pair_list.append(element)

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [48]:
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [50]:
#выделяем датафрейм co столбцами 'imdb_id', 'original_title', 'budget' по максимальному бюджету 'budget'
data[data.budget == data.budget.max()][['imdb_id', 'original_title', 'budget']] 

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


# 2. Какой из фильмов самый длительный (в минутах)?

In [51]:
answers['2'] = '1157 Gods and Generals (tt0279111)'

In [52]:
#выделяем датафрейм co столбцами 'imdb_id', 'original_title', 'runtime' по максимальной длительности фильма 'runtime'
data[data.runtime == data.runtime.max()][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
1157,tt0279111,Gods and Generals,214


# 3. Какой из фильмов самый короткий (в минутах)?





In [53]:
answers['3'] = '768 Winnie the Pooh (tt1449283)'

In [54]:
#выделяем датафрейм co столбцами 'imdb_id', 'original_title', 'runtime' по минимальной длительности фильма 'runtime'
data[data.runtime == data.runtime.min()][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


# 4. Какова средняя длительность фильмов?


In [55]:
answers['4'] = '110'

In [56]:
#вычисляем среднее по столбцу 'runtime'
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [57]:
answers['5'] = '107'

In [58]:
#вычисляем медиану по столбцу 'runtime'
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [59]:
answers['6'] = '239 Avatar (tt0499549)'

In [60]:
#выделяем датафрейм co столбцами 'imdb_id', 'original_title', 'profit' по максимальному значению прибыли 'profit'
data[data.profit == data.profit.max()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
239,tt0499549,Avatar,2544505847


# 7. Какой фильм самый убыточный? 

In [61]:
answers['7'] = '1245 The Lone Ranger (tt1210819)'

In [62]:
#выделяем датафрейм co столбцами 'imdb_id', 'original_title', 'profit' по минимальному значению прибыли 'profit'
data[data.profit == data.profit.min()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [63]:
answers['8'] = '1478'

In [64]:
#выделяем датафрейм, где значения столбца 'revenue' больше значений столбца 'budget'
#выводим количество уникальных идентификаторов фильмов 'imdb_id'
data[data.revenue > data.budget].imdb_id.nunique()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [65]:
answers['9'] = '599 The Dark Knight (tt0468569)'

In [105]:
#определяем новый датафрейм, содержащий фильмы вышедшие в 2008 году
data_year = data[data.release_year == 2008]

#из него выделяем датафрейм co столбцами 'imdb_id', 'original_title', 'revenue' по максимальным кассовым сборам 'revenue'
data_year[data_year.revenue == data_year.revenue.max()][['imdb_id', 'original_title', 'revenue']]

,imdb_id,original_title,revenue
599,tt0468569,The Dark Knight,1001921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [67]:
answers['10'] = '1245 The Lone Ranger (tt1210819)'

In [68]:
#определяем новый датафрейм, содержащий фильмы вышедшие с 2012 по 2014 года
data_period = data[(data.release_year >= 2012) & (data.release_year <= 2014)]

#из него выделяем датафрейм co столбцами 'imdb_id', 'original_title', 'profit' по минимальной прибыли 'profit'
data_period[data_period.profit == data_period. profit.min()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


# 11. Какого жанра фильмов больше всего?

In [69]:
answers['11'] = 'Drama'

In [70]:
record_list = []

#при помощи функци create list заполняем список 'record_list' жанрами
#количество элементов в списке равно тому, сколько раз жанры втсречаются в столбце 'genres'
#описание функци create list в ячейке "Предобработка"
data.genres.apply(create_list)

#создаём Series, где подсчитано количество каждого жанра в списке
s = pd.Series(Counter(record_list))

#выводим, отсортированный в порядке убывания, Series
s.sort_values(ascending=False)

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [71]:
answers['12'] = 'Drama'

In [72]:
#определяем новый датафрейм, содержащий значения больше нуля в столбце 'profit'
data_profit = data[data.profit > 0]

record_list = []

#при помощи функци create list заполняем список 'record_list' жанрами
#количество элементов в списке равно тому, сколько раз жанры втсречаются в столбце 'genres'
#описание функци create list в ячейке "Предобработка"
data_profit.genres.apply(create_list)

#создаём Series, где подсчитано количество каждого жанра в списке
s = pd.Series(Counter(record_list))

#выводим, отсортированный в порядке убывания, Series
s.sort_values(ascending=False)

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [73]:
answers['13'] = 'Peter Jackson'

In [74]:
#группируем данные по режиссерам 'director'
#сумируем значения кассовых сборов 'revenue' для каждого режиссера
#выводим полученные данные, отсортированные в порядке убывания
data.groupby(['director'])['revenue'].sum().sort_values(ascending=False)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Name: revenue, Length: 957, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [75]:
answers['14'] = 'Paul W.S. Anderson, Michael Bay'

In [76]:
#выделяем датафрейм содержащий значение 'Action' в столбце 'genres'
#выводим, сколько раз в выделенном датафрейме встречается каждый из режиссёров
data[data.genres.str.contains('Action', na=False)].director.value_counts()

Paul W.S. Anderson             7
Michael Bay                    7
Robert Rodriguez               6
Antoine Fuqua                  6
Ridley Scott                   6
                              ..
Ilya Naishuller                1
Ron Underwood                  1
Barry Levinson                 1
Brian Taylor|Mark Neveldine    1
Kerry Conran                   1
Name: director, Length: 345, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [108]:
answers['15'] = 'Chris Hemsworth'

In [107]:
"""Попытки определить новый датафрейм из основного, содержащий данные только за 2012 год, не увенчались успехом.
movie_release = data[data.release_year == 2012].copy. Возникает ошибка: 'function' object has no attribute 'cast'
Если вам известна причина ошибки, пожалуйста, напишите ответный комментарий."""

#создаем копию основного датафрейма
movie_release = data.copy()

#определяем датафрейм, содержащий данные только за 2012 год
movie_release = movie_release[movie_release.release_year == 2012]

#получаем списки с именами актёров и меняем на них значения столбца 'cast'
movie_release['cast'] = movie_release.cast.str.split('|')

#преобразуем каждый элемент всех списков в отдельную строку
movie_release = movie_release.explode('cast')

#группируем данные по именам актёров
#вычисляем сумму кассовых сборов по столбцу 'revenue' для каждого актёра
#выводим, отсортированные в порядке убывания, данные
movie_release.groupby(['cast'])['revenue'].sum().sort_values(ascending=False)

cast
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Chris Evans          1519557910
Robert Downey Jr.    1519557910
                        ...    
Jason Bateman           3428048
Danny Huston            2106557
Sami Gayle              2106557
Josh Lucas              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [78]:
answers['16'] = 'Matt Damon'

In [79]:
#определяем новый датафрейм со значениями больше среднего по бюджету 'budget'
big_budget = data[data.budget > data.budget.mean()]
record_list = []

#при помощи функци create list заполняем список 'record_list' именами актеров из столбца 'cast'
#количество элементов в списке равно тому, сколько раз имена актеров втсречаются в столбце 'cast'
#описание функци create list в ячейке "Предобработка"
big_budget.cast.apply(create_list)

#создаём Series, где подсчитано количество имен каждого актера в списке
s = pd.Series(Counter(record_list))

#выводим, отсортированный в порядке убывания, Series
s.sort_values(ascending=False)

Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Tom Cruise           15
Samuel L. Jackson    15
                     ..
Marsha Thomason       1
Jennifer Tilly        1
Eileen Atkins         1
Jenna Elfman          1
Piper Perabo          1
Length: 1505, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [80]:
answers['17'] = 'Action'

In [81]:
#определяем новый датафрейм содержащий значение 'Nicolas Cage' в столбце 'cast'
cage = data[data.cast.str.contains('Nicolas Cage')]
record_list = []

#при помощи функци create list заполняем список 'record_list' жанрами
#количество элементов в списке равно тому, сколько раз жанры втсречаются в столбце 'genres'
#описание функци create list в ячейке "Предобработка"
cage.genres.apply(create_list)

#создаём Series, где подсчитано количество каждого жанра в списке
s = pd.Series(Counter(record_list))

#выводим, отсортированный в порядке убывания, Series
s.sort_values(ascending=False)

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Family              3
Animation           3
Mystery             3
History             2
Romance             1
Horror              1
War                 1
dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [82]:
answers['18'] = '925 K-19: The Widowmaker (tt0267626)'

In [83]:
#определяем новый датафрейм содержащий значение 'Paramount Pictures' в столбце 'production_companies'
paramount = data[data.production_companies.str.contains('Paramount Pictures')]

#выделяем датафрейм co столбцами 'imdb_id', 'original_title', 'profit' по минимальной прибыли 'profit'
paramount[paramount.profit == paramount.profit.min()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
925,tt0267626,K-19: The Widowmaker,-64831034


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [84]:
answers['19'] = '2015'

In [85]:
#группируем данные по году выпуска фильмов 'release_year'
#суммируем кассовые сборы 'revenue' для каждого года
#выводим, отсортированные в порядке убывания, данные
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [86]:
answers['20'] = '2014'

In [87]:
#определяем новый датафрейм содержащий значение 'Warner Bros' в столбце 'production_companies'
warner = data[data.production_companies.str.contains('Warner Bros')]

#группируем данные по году выпуска фильмов 'release_year'
#суммируем значения прибыли 'profit' по годам
#выводим, отсортированные в порядке убывания, данные
warner.groupby(['release_year'])['profit'].sum().sort_values(ascending=False)

release_year
2014    2295464519
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1855493377
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1343545668
2012    1258020056
2002    1022709901
2015     870368348
2006     620170743
2000     452631386
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [88]:
answers['21'] = 'September'

In [89]:
#определяем новый датафрейм и присваиваем ему копию основного датафрейма
release_month = data.copy()

#создаем новый столбец 'month', в котором содержится месяц релиза каждого фильма
release_month['month'] = data.release_date.apply(lambda x: re.search('\d+', x).group())

#группируем данные по месяцу релиза
#подсчитываем количество фильмов по месяцам
#выводим, отсортированные в порядке убывания, данные
release_month.groupby(['month'])['imdb_id'].count().sort_values(ascending=False)

month
9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [90]:
answers['22'] = '450'

In [91]:
#используем созданный на 21-м шаге датафрейм
#выделяем датафрейм по столбцу 'month' со значениями, равными летним месяцам
#подсчитываем количество идентификаторов фильмов
release_month[(release_month.month == '6') | (release_month.month == '7') | (release_month.month == '8')].imdb_id.count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [92]:
answers['23'] = 'Peter Jackson'

In [93]:
#используем созданный на 21-м шаге датафрейм
#выделяем датафрейм по столбцу 'month' со значениями, равными зимним месяцам
#выводим сколько раз встречается каждый режиссёр в столбце 'director
release_month[(release_month.month == '12') | (release_month.month == '1') | (release_month.month == '2')].director.value_counts()

Peter Jackson        7
Clint Eastwood       6
Steven Soderbergh    6
Martin Scorsese      4
Nancy Meyers         4
                    ..
Will Gluck           1
NimrÃ³d Antal        1
Steve Pink           1
John Erick Dowdle    1
Daniel Barnz         1
Name: director, Length: 332, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [94]:
answers['24'] = 'Four By Two Productions'

In [95]:
#определяем новый датафрейм со столбцами 'original_title' и 'production_companies' из основного датафрейма
len_origin_title = data[['original_title', 'production_companies']].copy()

#создаем новый столбец 'length_title', содержащий количество символов в названии каждого фильма
len_origin_title['length_title'] = data.original_title.apply(lambda x: len(x))

#получаем списки с названиями студий и меняем на них значения столбца 'production_companies'
len_origin_title['production_companies'] = len_origin_title.production_companies.str.split('|')

#преобразуем каждый элемент всех списков в отдельную строку
len_origin_title = len_origin_title.explode('production_companies')

#группируем данные по названиям студий
#вычисляем среднюю длину названий фильмов по столбцу 'length_title' для каждой студии
#выводим, отсортированные в порядке убывания, данные
len_origin_title.groupby(['production_companies'])['length_title'].mean().sort_values(ascending=False)

production_companies
Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Museum Canada Productions     46.0
Polsky Films                  46.0
                              ... 
Everest Entertainment          3.0
Berlanti Productions           3.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Global Entertainment Group     2.0
Name: length_title, Length: 1771, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [96]:
answers['25'] = 'Midnight Picture Show'

In [97]:
#определяем новый датафрейм со столбцами 'production_companies' и 'overview' из основного датафрейма
movie_overview = data[['production_companies', 'overview']].copy()

#создаем новый столбец 'len_overview', содержащий количество слов в описании каждого фильма
movie_overview['len_overview'] = movie_overview.overview.apply(lambda x: len(x.split()))

#получаем списки с названиями студий и меняем на них значения столбца 'production_companies'
movie_overview['production_companies'] = movie_overview.production_companies.str.split('|')

#преобразуем каждый элемент всех списков в отдельную строку
movie_overview = movie_overview.explode('production_companies')

#группируем данные по названиям студий
#вычисляем среднее количество слов по столбцу 'len_overview' для каждой студии
#выводим, отсортированные в порядке убывания, данные
movie_overview.groupby(['production_companies'])['len_overview'].mean().sort_values(ascending=False)

production_companies
Midnight Picture Show                    175.0
Room 9 Entertainment                     161.0
98 MPH Productions                       159.0
Heineken Branded Entertainment           159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
London Boulevard                          13.0
Phantom Four                              13.0
Henceforth                                13.0
Empire Pictures                           11.0
Motion Picture Corporation of America     11.0
Name: len_overview, Length: 1771, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [98]:
answers['26'] = 'Inside Out, Room, Interstellar, Guardians of the Galaxy, The Imitation Game, Gone Girl, The Grand Budapest Hotel, Inception, The Dark Knight, The Pianist, The Lord of the Rings: The Return of the King, The Wolf of Wall Street, 12 Years a Slave, Memento'

In [99]:
#выделяем датафрейм со столбцами 'imdb_id', 'original_title', 'vote_average' по стобцу 'vote_average'
#из столбца 'vote_average' берутся значения, превышающие то значение, которое соответствует квантилю 99%
data[data.vote_average > data.vote_average.quantile(0.99)][['imdb_id', 'original_title', 'vote_average']]

,imdb_id,original_title,vote_average
9,tt2096673,Inside Out,8.0
34,tt3170832,Room,8.0
118,tt0816692,Interstellar,8.0
119,tt2015381,Guardians of the Galaxy,7.9
125,tt2084970,The Imitation Game,8.0
128,tt2267998,Gone Girl,7.9
138,tt2278388,The Grand Budapest Hotel,7.9
370,tt1375666,Inception,7.9
599,tt0468569,The Dark Knight,8.1
872,tt0253474,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [100]:
answers['27'] = 'Daniel Radcliffe, Rupert Grint, Emma Watson'

In [110]:
pair_list = []

#при помощи функци create list2 заполняется список 'pair_list' всеми возможными парами имен актёров, снимавшихся в одном фильме
#описание функци create list2 в ячейке "Предобработка"
data.cast.apply(create_list2) 

#создаём Series, где подсчитано количество каждой пары имен
s = pd.Series(Counter(actor_list))

#выводим первые 10 значений Series, отсортированные в порядке убывания
s.sort_values(ascending=False).head(10)

Daniel Radcliffe, Rupert Grint       8
Rupert Grint, Daniel Radcliffe       8
Daniel Radcliffe, Emma Watson        8
Emma Watson, Rupert Grint            8
Rupert Grint, Emma Watson            8
Emma Watson, Daniel Radcliffe        8
Owen Wilson, Ben Stiller             6
Helena Bonham Carter, Johnny Depp    6
Ben Stiller, Owen Wilson             6
Johnny Depp, Helena Bonham Carter    6
dtype: int64

# Submission

In [109]:
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157 Gods and Generals (tt0279111)',
 '3': '768 Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': '239 Avatar (tt0499549)',
 '7': '1245 The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': '599 The Dark Knight (tt0468569)',
 '10': '1245 The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Paul W.S. Anderson, Michael Bay',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': '925 K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, Room, Interstellar, Guardians of the Galaxy, The Imitation Game, Gone Girl, The Grand Budapest Hotel, Inception, The Dark Knight, The Pianist, The Lord of the Rings: The Return of the King, The Wolf of Wall Street, 12 Years a Slave, Memento',
 '27': 'Daniel Radcli

In [103]:
len(answers)

27